#Momento de Retroalimentación: Módulo 1 Utilización, procesamiento y visualización de grandes volúmenes de datos (Portafolio Análisis)

Para esta actividad, se hara la prediccion sobre si una persona es propensa a tener un ataque cardiaco segun sus datos medicos

##Inicio de sesion en spark

In [1]:
#Bibliotecas para poder trabajar con Spark
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.2//spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
#Configuración de Spark con Python
!pip install -q findspark
!pip install pyspark

#Estableciendo variable de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

#Buscando e inicializando la instalación de Spark
import findspark
findspark.init()
findspark.find()

#Probando PySparl
from pyspark.sql import SparkSession

spark_Act1 = SparkSession \
       .builder \
       .appName("Act1") \
       .getOrCreate()

spark_Act1

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [96.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ge

## Acceso a datos de Google Drive

In [2]:
from google.colab import drive

drive.mount("/content/gdrive") 

Mounted at /content/gdrive


In [5]:
%cd "/content/gdrive/MyDrive/Bloque 2 IA/Modulo 1 BD/Entregable"
!ls

/content/gdrive/MyDrive/Bloque 2 IA/Modulo 1 BD/Entregable
Untitled0.ipynb  Wine.csv


##Obtencion del dataset 

Dataset obtenido de kaggle [Heart Attack Analysis](https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset?select=heart.csv)

In [117]:
df = spark_Act1.read.csv('/content/gdrive/MyDrive/Bloque 2 IA/Modulo 1 BD/Entregable/heart.csv', header=True, inferSchema=True)
df

DataFrame[age: int, sex: int, cp: int, trtbps: int, chol: int, fbs: int, restecg: int, thalachh: int, exng: int, oldpeak: double, slp: int, caa: int, thall: int, output: int]

In [118]:
#Visualizamos el dataset para conocer los datos a tratar
df.show()

+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
|age|sex| cp|trtbps|chol|fbs|restecg|thalachh|exng|oldpeak|slp|caa|thall|output|
+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
| 63|  1|  3|   145| 233|  1|      0|     150|   0|    2.3|  0|  0|    1|     1|
| 37|  1|  2|   130| 250|  0|      1|     187|   0|    3.5|  0|  0|    2|     1|
| 41|  0|  1|   130| 204|  0|      0|     172|   0|    1.4|  2|  0|    2|     1|
| 56|  1|  1|   120| 236|  0|      1|     178|   0|    0.8|  2|  0|    2|     1|
| 57|  0|  0|   120| 354|  0|      1|     163|   1|    0.6|  2|  0|    2|     1|
| 57|  1|  0|   140| 192|  0|      1|     148|   0|    0.4|  1|  0|    1|     1|
| 56|  0|  1|   140| 294|  0|      0|     153|   0|    1.3|  1|  0|    2|     1|
| 44|  1|  1|   120| 263|  0|      1|     173|   0|    0.0|  2|  0|    3|     1|
| 52|  1|  2|   172| 199|  1|      1|     162|   0|    0.5|  2|  0|    3|     1|
| 57|  1|  2|   150| 168|  0

In [120]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trtbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalachh: integer (nullable = true)
 |-- exng: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slp: integer (nullable = true)
 |-- caa: integer (nullable = true)
 |-- thall: integer (nullable = true)
 |-- output: integer (nullable = true)



## Data preparation

Creacion del Vector Assembler con los datos relevantes del Dataset

In [121]:
from pyspark.ml.feature import VectorAssembler

cols = ['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs',
        'restecg', 'thalachh', 'exng', 'oldpeak', 
        'slp', 'caa', 'thall']

assembler = VectorAssembler(
    inputCols=cols,
    outputCol="features")

transformed = assembler.transform(df)

#Creamos el dataset final con el vector assembler y los outputs
final_data = transformed.select("features", "output")
final_data.show()

+--------------------+------+
|            features|output|
+--------------------+------+
|[63.0,1.0,3.0,145...|     1|
|[37.0,1.0,2.0,130...|     1|
|[41.0,0.0,1.0,130...|     1|
|[56.0,1.0,1.0,120...|     1|
|[57.0,0.0,0.0,120...|     1|
|[57.0,1.0,0.0,140...|     1|
|[56.0,0.0,1.0,140...|     1|
|[44.0,1.0,1.0,120...|     1|
|[52.0,1.0,2.0,172...|     1|
|[57.0,1.0,2.0,150...|     1|
|[54.0,1.0,0.0,140...|     1|
|[48.0,0.0,2.0,130...|     1|
|[49.0,1.0,1.0,130...|     1|
|[64.0,1.0,3.0,110...|     1|
|[58.0,0.0,3.0,150...|     1|
|[50.0,0.0,2.0,120...|     1|
|[58.0,0.0,2.0,120...|     1|
|[66.0,0.0,3.0,150...|     1|
|[43.0,1.0,0.0,150...|     1|
|[69.0,0.0,3.0,140...|     1|
+--------------------+------+
only showing top 20 rows



In [122]:
# Split en train y test
(df_train, df_test) = final_data.randomSplit([0.7, 0.3], seed=42)

##Creación del modelo

In [123]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'output', maxIter=10)
lrModel = lr.fit(df_train)

## Predicción del modelo

In [124]:
prediction_result = lrModel.evaluate(df_test)
prediction_result.predictions.show()

+--------------------+------+--------------------+--------------------+----------+
|            features|output|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|(13,[0,1,3,4,7,10...|     1|[-1.5294992082306...|[0.17806696941440...|       1.0|
|(13,[0,2,3,4,7,10...|     1|[-4.2782986501539...|[0.01367659054740...|       1.0|
|(13,[0,3,4,6,7,10...|     1|[-3.5823965274157...|[0.02705655825811...|       1.0|
|(13,[0,3,4,7,8,10...|     1|[-0.5210565014791...|[0.37260522151029...|       1.0|
|(13,[0,3,4,7,9,10...|     1|[-0.6947875105924...|[0.33296891525768...|       1.0|
|(13,[0,3,4,7,9,11...|     0|[2.75572945338308...|[0.94023611344024...|       0.0|
|(13,[0,3,4,7,9,11...|     0|[5.87436588741898...|[0.99719730134606...|       0.0|
|(13,[0,3,4,7,10,1...|     0|[-2.6844787018020...|[0.06389546874026...|       1.0|
|[34.0,0.0,1.0,118...|     1|[-4.0812853459174...|[0.01660535385609...|       1.0|
|[35

/content/spark-3.2.2-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


## Metricas obtenidas por el modelo

Accuracy del modelo en train

In [125]:
lrModel.summary.accuracy

0.84375

Accuracy del modelo en test

In [126]:
prediction_result.accuracy

0.8481012658227848

Area under ROC del modelo en test

In [127]:
prediction_result.areaUnderROC

0.9208776595744681